##### Copyright 2019 The TensorFlow Authors.

In [14]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [15]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [16]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [17]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [18]:
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[ 0.32089594, -0.13868068, -0.12980568, -0.0331073 ,  0.18877307,
         0.13736251,  0.23968783, -0.07974359,  0.12401713, -0.03766727]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [19]:
tf.nn.softmax(predictions).numpy()

array([[0.12835282, 0.08106135, 0.08178397, 0.09008733, 0.11246702,
        0.10683114, 0.11834152, 0.08598246, 0.1054149 , 0.08967747]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [20]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [21]:
loss_fn(y_train[:1], predictions).numpy()

2.2365057

In [22]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [23]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 28s 460us/sample - loss: 0.2636 - accuracy: 0.9204
Epoch 2/5
60000/60000 [==============================] - 22s 374us/sample - loss: 0.1168 - accuracy: 0.9656
Epoch 3/5
60000/60000 [==============================] - 22s 372us/sample - loss: 0.0844 - accuracy: 0.9748
Epoch 4/5
60000/60000 [==============================] - 22s 373us/sample - loss: 0.0681 - accuracy: 0.9793
Epoch 5/5
60000/60000 [==============================] - 23s 389us/sample - loss: 0.0546 - accuracy: 0.9836


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [24]:
model.evaluate(x_test,  y_test, verbose=2)

10000/1 - 9s - loss: 0.0466 - accuracy: 0.9743


[0.09301762199170771, 0.9743]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [25]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [26]:
probability_model(x_test[:5])

<tf.Tensor: id=60412, shape=(5, 10), dtype=float32, numpy=
array([[3.1073078e-11, 1.3745777e-06, 6.2765016e-06, 4.0511968e-06,
        1.0024373e-06, 1.4845006e-08, 1.0835309e-11, 9.9998474e-01,
        2.4758352e-08, 2.5309080e-06],
       [3.3817455e-14, 2.7057859e-10, 1.0000000e+00, 5.6155327e-11,
        5.7730195e-14, 1.6261305e-13, 3.6212505e-15, 1.4337010e-09,
        4.8017538e-11, 8.2485842e-17],
       [2.3435491e-09, 9.9998653e-01, 3.7788303e-07, 2.6879994e-09,
        2.5133815e-07, 9.9675583e-07, 3.5886939e-07, 2.4666869e-07,
        1.1227650e-05, 1.4903019e-08],
       [9.9991930e-01, 7.7362683e-10, 3.9146694e-06, 5.9938725e-07,
        4.9709621e-07, 3.6198750e-07, 1.1986819e-05, 2.8468219e-07,
        8.4962045e-07, 6.2177402e-05],
       [3.3856047e-08, 4.8070799e-08, 9.0399617e-08, 6.0793588e-09,
        9.9813521e-01, 5.6780134e-08, 4.8160743e-08, 3.8814410e-06,
        2.1610538e-06, 1.8586046e-03]], dtype=float32)>